In [1]:
from io import StringIO
import pandas as pd
from sqlalchemy import create_engine, text
engine = create_engine('sqlite://', echo=False)
import sqlite3
import contextlib

def df_to_database(df, name):
    """
       Deze functie maakt een database van een dataframe.
       Parameters:
                  df  = de dataframe waarvan je een database tabel wil maken.
                         
                  name = De naam die je de database wil geven.
                         Zorg ervoor dat deze parameter een string is.
    """
    df.to_sql(name, con=engine)
    
    display(df)

def string_to_df(string):
    """
       Deze functie maakt van een string een dataframe,
       zolang het standaard CSV formaat wordt aangehouden.
       Parameters:
                  string = de string waarvan je een dataframe wil maken.
    """
    return pd.read_csv(StringIO(string))

def show_query(query):
    """
       Deze functie print de output van je SQL query uit.
       Parameters:
                  query = Je SQL query.
                          Zorg ervoor dat deze parameter een string is.
    """

    query_output = engine.execute(query).fetchall()
    column_names = engine.execute(query)._metadata.keys
    return pd.DataFrame(query_output, columns = column_names)

In [2]:
sql_connect = sqlite3.connect('database.sqlite')
cursor = sql_connect.cursor()

In [3]:
query_r = "SELECT * FROM sqlite_sequence" # read query

sql_connect = sqlite3.connect('database.sqlite')
cursor = sql_connect.cursor()
df = pd.read_sql_query(query_r,sql_connect)
df

,name,seq
0,Team,103916
1,Country,51958
2,League,51958
3,Match,51958
4,Player,11075
5,Player_Attributes,183978
6,Team_Attributes,1458


In [4]:
import contextlib
@contextlib.contextmanager
def db_read_only(file):
    sql_connect = sqlite3.connect('file:'+file+'?mode=ro',uri=True)
    cursor = sql_connect.cursor()
    yield sql_connect
    sql_connect.commit()
    sql_connect.close() 

In [5]:
file = 'database.sqlite'
with db_read_only(file) as ro:
    data = pd.read_sql_query(query_r, ro)

In [6]:
data['name']

0                 Team
1              Country
2               League
3                Match
4               Player
5    Player_Attributes
6      Team_Attributes
Name: name, dtype: object

In [7]:
df_list = []
with db_read_only(file) as ro:
    for table in data['name']:
        query_r = 'SELECT * FROM ' + str(table)
        data = pd.read_sql_query(query_r, ro)
        df_list.append(data)
df_list[0]

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB
...,...,...,...,...,...
294,49479,10190,898.0,FC St. Gallen,GAL
295,49837,10191,1715.0,FC Thun,THU
296,50201,9777,324.0,Servette FC,SER
297,50204,7730,1862.0,FC Lausanne-Sports,LAU


In [8]:
team = df_list[0]
country = df_list[1]
league = df_list[2]
match = df_list[3]
player = df_list[4]
player_attributes = df_list[5]
team_attributes = df_list[6]

In [30]:
match = match.merge(team, left_on = "home_team_api_id", right_on = "team_api_id", how = "left", suffixes = ["_match", "_team"])
match

,id_match,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,id_league,team_api_id_match,team_fifa_api_id_match,team_long_name_match,team_short_name_match,id,team_api_id_team,team_fifa_api_id_team,team_long_name_team,team_short_name_team
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,1,9987,673.0,KRC Genk,GEN,1,9987,673.0,KRC Genk,GEN
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3,10000,15005.0,SV Zulte-Waregem,ZUL,3,10000,15005.0,SV Zulte-Waregem,ZUL
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,5,9984,1750.0,KSV Cercle Brugge,CEB,5,9984,1750.0,KSV Cercle Brugge,CEB
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7,9991,674.0,KAA Gent,GEN,7,9991,674.0,KAA Gent,GEN
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,9,7947,NaN,FCV Dender EH,DEN,9,7947,NaN,FCV Dender EH,DEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,49479,10190,898.0,FC St. Gallen,GAL,49479,10190,898.0,FC St. Gallen,GAL
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,49122,9824,286.0,FC Vaduz,VAD,49122,9824,286.0,FC Vaduz,VAD
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,49115,9956,322.0,Grasshopper Club Zürich,GRA,49115,9956,322.0,Grasshopper Club Zürich,GRA
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,51606,7896,NaN,Lugano,LUG,51606,7896,NaN,Lugano,LUG


In [32]:
df_to_database(match, 'match_sql1')
print('Tabel 1: match_sql')

,id_match,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,id_league,team_api_id_match,team_fifa_api_id_match,team_long_name_match,team_short_name_match,id,team_api_id_team,team_fifa_api_id_team,team_long_name_team,team_short_name_team
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,1,9987,673.0,KRC Genk,GEN,1,9987,673.0,KRC Genk,GEN
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3,10000,15005.0,SV Zulte-Waregem,ZUL,3,10000,15005.0,SV Zulte-Waregem,ZUL
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,5,9984,1750.0,KSV Cercle Brugge,CEB,5,9984,1750.0,KSV Cercle Brugge,CEB
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7,9991,674.0,KAA Gent,GEN,7,9991,674.0,KAA Gent,GEN
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,9,7947,NaN,FCV Dender EH,DEN,9,7947,NaN,FCV Dender EH,DEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,49479,10190,898.0,FC St. Gallen,GAL,49479,10190,898.0,FC St. Gallen,GAL
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,49122,9824,286.0,FC Vaduz,VAD,49122,9824,286.0,FC Vaduz,VAD
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,49115,9956,322.0,Grasshopper Club Zürich,GRA,49115,9956,322.0,Grasshopper Club Zürich,GRA
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,51606,7896,NaN,Lugano,LUG,51606,7896,NaN,Lugano,LUG


Tabel 1: match_sql


In [33]:
query = """

SELECT COUNT(match_api_id) AS amount_of_games, season
FROM match_sql1
WHERE (home_team_api_id = 9906 OR away_team_api_id = 9906)
GROUP BY season

"""

show_query(query)

,amount_of_games,season
0,38,2008/2009
1,38,2009/2010
2,38,2010/2011
3,38,2011/2012
4,38,2012/2013
5,38,2013/2014
6,38,2014/2015
7,38,2015/2016


In [34]:
query = """

SELECT COUNT(match_api_id) AS amount_of_games, season
FROM match_sql1
WHERE (home_team_api_id = 9906 OR away_team_api_id = 9906)
AND date LIKE '%2010%'
GROUP BY season

"""

show_query(query)

,amount_of_games,season
0,23,2009/2010
1,16,2010/2011


In [76]:
query = """

WITH homepoints AS (SELECT home_team_api_id AS team, season,
                           CASE
                             WHEN home_team_goal > away_team_goal THEN 3
                             WHEN home_team_goal = away_team_goal THEN 1
                             ELSE 0
                           END AS points
                        FROM match_sql1
                        GROUP BY season),
      awaypoints AS (SELECT away_team_api_id AS team,season,
                           CASE
                             WHEN away_team_goal > home_team_goal THEN 3
                             WHEN away_team_goal = home_team_goal THEN 1
                             ELSE 0
                           END AS points
                        FROM match_sql1
                        GROUP BY season),
      allpoints AS (SELECT team, points FROM homepoints, season
                    UNION ALL
                    SELECT team, points FROM awaypoints, season)

SELECT m.home_team_api_id AS team, m.team_long_name_team AS teamname, m.season, SUM(ap.points) AS total_points
FROM match_sql1 AS m
LEFT OUTER JOIN allpoints as ap
ON ap.team = m.home_team_api_id
WHERE league_id = 21518 AND season = '2008/2009'
GROUP BY m.home_team_api_id, m.team_long_name_team, m.season
ORDER BY total_points DESC, m.home_team_api_id             

"""

show_query(query)

,team,teamname,season,total_points
0,8634,FC Barcelona,2008/2009,14155
1,8633,Real Madrid CF,2008/2009,13680
2,9906,Atlético Madrid,2008/2009,10640
3,10267,Valencia CF,2008/2009,9500
4,8302,Sevilla FC,2008/2009,9158
5,8315,Athletic Club de Bilbao,2008/2009,8303
6,10205,Villarreal CF,2008/2009,7733
7,9864,Málaga CF,2008/2009,7524
8,8558,RCD Espanyol,2008/2009,6916
9,8305,Getafe CF,2008/2009,6707


In [73]:
query = """
WITH homepoints AS (SELECT home_team_api_id AS team,
                           CASE
                             WHEN home_team_goal > away_team_goal THEN 3
                             WHEN home_team_goal = away_team_goal THEN 1
                             ELSE 0
                           END AS points
                        FROM match_sql1),
      awaypoints AS (SELECT away_team_api_id AS team,
                           CASE
                             WHEN away_team_goal > home_team_goal THEN 3
                             WHEN away_team_goal = home_team_goal THEN 1
                             ELSE 0
                           END AS points
                        FROM match_sql1)


SELECT team, points FROM homepoints
UNION ALL
SELECT team, points FROM awaypoints
                    

"""
show_query(query)

,team,points
0,9987,1
1,10000,1
2,9984,0
3,9991,3
4,7947,0
...,...,...
51953,10191,0
51954,10199,3
51955,10179,0
51956,10243,1


In [86]:
query = """

WITH homepoints AS (SELECT home_team_api_id AS team,
                           CASE
                            WHEN home_team_goal > away_team_goal THEN 3
                            WHEN home_team_goal = away_team_goal THEN 1
                                ELSE 0
                               END AS points
                            FROM match_sql),
      awaypoints AS (SELECT away_team_api_id AS team,
                           CASE
                            WHEN away_team_goal > home_team_goal THEN 3
                            WHEN away_team_goal = home_team_goal THEN 1
                                ELSE 0
                               END AS points
                            FROM match_sql),
      allpoints AS (SELECT team, points FROM homepoints
                    INTERSECT
                    SELECT team, points FROM awaypoints)

SELECT m.home_team_api_id AS team, m.team_long_name_team AS teamname, m.season, COALESCE(SUM(ap.points), 0) AS total_points
FROM match_sql1 AS m
LEFT OUTER JOIN allpoints as ap
ON ap.team = m.home_team_api_id
WHERE league_id = 21518 AND season = '2008/2009'
GROUP BY m.home_team_api_id, m.team_long_name_team, m.season
ORDER BY COALESCE(SUM(POINTS), 0) DESC, m.home_team_api_id             

"""

show_query(query)

,team,teamname,season,total_points
0,8302,Sevilla FC,2008/2009,76
1,8305,Getafe CF,2008/2009,76
2,8315,Athletic Club de Bilbao,2008/2009,76
3,8371,CA Osasuna,2008/2009,76
4,8388,CD Numancia,2008/2009,76
5,8479,RC Recreativo,2008/2009,76
6,8558,RCD Espanyol,2008/2009,76
7,8603,Real Betis Balompié,2008/2009,76
8,8633,Real Madrid CF,2008/2009,76
9,8634,FC Barcelona,2008/2009,76


In [85]:
query = """
WITH homepoints AS (SELECT home_team_api_id AS team, season,
                           CASE
                             WHEN home_team_goal > away_team_goal THEN 3
                             WHEN home_team_goal = away_team_goal THEN 1
                             ELSE 0
                           END AS points
                        FROM match_sql1
                        GROUP BY season),
      awaypoints AS (SELECT away_team_api_id AS team,season,
                           CASE
                             WHEN away_team_goal > home_team_goal THEN 3
                             WHEN away_team_goal = home_team_goal THEN 1
                             ELSE 0
                           END AS points
                        FROM match_sql1
                        GROUP BY season),
      allpoints AS (SELECT homepoints.team, homepoints.points, homepoints.season 
                    FROM homepoints
                    JOIN awaypoints
                    ON homepoints.season = awaypoints.season)
SELECT * 
FROM allpoints
                    
"""
show_query(query)

,team,points,season
0,9987,1,2008/2009
1,9985,1,2009/2010
2,9997,3,2010/2011
3,1773,3,2011/2012
4,8571,1,2012/2013
5,8475,3,2013/2014
6,9985,3,2014/2015
7,9997,3,2015/2016
